### 4-12

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris(as_frame = True)
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

In [ ]:
from sklearn.model_selection import train_test_split
iris.data.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [ ]:
import pandas as pd
X = iris.data.iloc[:,[2,3]]
ones = pd.DataFrame(np.ones((X.shape[0], 1)), columns=["bias"])
X_with_bias = pd.concat([ones,X], axis = 1)
y = iris.target

In [ ]:
#train_test_splitを実装
def train_test_split(X, y, test_size, random_state):
  if random_state != None:
    random_state = 2042

  #インデックスをシャッフル
  indice = np.arange(X.shape[0])
  np.random.shuffle(indice)

  test_size = int(test_size * X.shape[0])
  train_indice = indice[:-test_size]
  test_indice = indice[-test_size:]

  X_train = X.iloc[train_indice]
  X_test = X.iloc[test_indice]
  y_train = y.iloc[train_indice]
  y_test = y.iloc[test_indice]

  return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(X_with_bias, y, 0.2, 2042)
val_size = 0.2
val_size = int(val_size * X_train.shape[0])
X_train = X_train.iloc[:-val_size]
X_val = X_train.iloc[-val_size:]
y_train = y_train.iloc[:-val_size]
y_val = y_train.iloc[-val_size:]

#Softmax関数を学習させるためにtarget変数をone-hot encoding
def one_hot(y):
  n_class = y.max() + 1
  m = len(y)
  ohe_y = np.zeros((m,n_class))
  ohe_y[np.arange(m), y] = 1
  return ohe_y

y_train_ohe = one_hot(y_train)
y_test_ohe = one_hot(y_test)
y_val_ohe = one_hot(y_val)

In [ ]:
#softmax分類器を定義
def softmax(logits):
  exps = np.exp(logits)
  exps_sum = np.sum(exps, axis = 1, keepdims = True)
  return exps / exps_sum


In [ ]:
n_inputs = X_train.shape[1]
n_outputs = len(np.unique(y_train))

In [ ]:
Theta = np.random.randn(n_inputs, n_outputs)

logits = X_train.dot(Theta)
logits = logits.to_numpy()
print(X_train.shape)
softmax(logits).shape

(96, 3)


(96, 3)

In [ ]:
#ここからモデルの学習開始．損失関数を定義して，バッチ勾配降下法を実装する
eta = 0.2
n_iterations = 5001
epsilon = 1e-7
m = len(X_train)

Theta = np.random.randn(n_inputs, n_outputs)

for iterations in range(n_iterations):
  logits = X_train.dot(Theta)
  logits = logits.to_numpy()
  y_prob = softmax(logits)
  if iterations % 200 == 0:
    loss = -np.mean(np.sum(y_train_ohe * np.log(y_prob + epsilon), axis=1))
    print(iterations, loss)

  error = y_prob - y_train_ohe
  gradients = 1/m * X_train.T.dot(error)
  Theta = Theta - gradients * eta

0 2.057644959111115
200 0.3907415739066922
400 0.30152442838001353
600 0.2572131426422736
800 0.22868763220347632
1000 0.2082737377640196
1200 0.1927758711502757
1400 0.18054512937498188
1600 0.17061835083718946
1800 0.16238590237421865
2000 0.15543955458345238
2200 0.14949420806591965
2400 0.1443441015517323
2600 0.13983672533568436
2800 0.13585650526812085
3000 0.13231419101160763
3200 0.1291397286304055
3400 0.126277342110609
3600 0.12368205961405537
3800 0.12131721014697704
4000 0.11915258726408688
4200 0.11716308065458288
4400 0.11532764186628904
4600 0.11362849252316243
4800 0.11205051109840332
5000 0.11058075290692966


In [ ]:
Theta

,0,1,2
bias,7.290750,0.258278,-6.363529
petal length (cm),-1.730988,0.835278,0.457199
petal width (cm),-1.546812,-1.408223,3.693623


In [ ]:
#Early stoppingを実装したversion
eta = 0.02
n_iterations = 10001
epsilon = 1e-7
m = len(X_train)
best_loss = np.infty

Theta = np.random.randn(n_inputs, n_outputs)

for iterations in range(n_iterations):
  logits = X_train.dot(Theta)
  logits = logits.to_numpy()
  y_prob = softmax(logits)
  if iterations % 200 == 0:
    loss = -np.mean(np.sum(y_train_ohe * np.log(y_prob + epsilon), axis=1))
    print(iterations, loss)

  error = y_prob - y_train_ohe
  gradients = 1/m * X_train.T.dot(error)
  Theta = Theta - gradients * eta
  #ここから変更
  val_logits = X_val.dot(Theta)
  val_logits = val_logits.to_numpy()
  y_prob_val = softmax(val_logits)
  val_error = y_prob_val - y_val_ohe
  val_gradients = X_val.T.dot(val_error)
  loss_val = -np.mean(np.sum(y_val_ohe * np.log(y_prob_val + epsilon), axis = 1))

  if best_loss > loss_val:
    best_loss = loss_val
  else:
    print(iterations - 1, best_loss)
    print(iterations, loss_val, "Early stopping")
    break

0 1.87176865267997
200 0.9539145023229659
400 0.7775016684889703
600 0.6658554602911746
800 0.5916144887730397
1000 0.5391159242726994
1200 0.4998838963549748
1400 0.4692220931246343
1600 0.4443976002699946
1800 0.4237341142804876
2000 0.40615198953383547
2200 0.39092528650830394
2400 0.37754738619537703
2600 0.3656532400134111
2800 0.35497248123812636
3000 0.3453000684690024
3200 0.3364772791647588
3400 0.3283790308213659
3600 0.3209051948897991
3800 0.313974502994049
4000 0.30752018005649395
4200 0.30148675485126525
4400 0.2958276903665653
4600 0.29050359592663205
4800 0.285480859327666
5000 0.28073058700575726
5200 0.2762277733648944
5400 0.27195064283173986
5600 0.2678801236704631
5800 0.26399942342002697
6000 0.2602936835089626
6200 0.25674969614099324
6400 0.25335567058208097
6600 0.2501010389566103
6800 0.2469762938795734
7000 0.2439728519224245
7200 0.2410829381800421
7400 0.23829948817972343
7600 0.23561606412560535
7800 0.23302678305804625
8000 0.23052625496737858
8200 0.2281

In [ ]:
#validation
logits = X_test.dot(Theta)
logits = logits.to_numpy()
y_prob = softmax(logits)
y_predict = np.argmax(y_prob, axis = 1)

score = np.mean(y_predict == y_test)
score

0.9666666666666667